# Notebook for scraping different regions

This notebook outlines how to scrape high level games from different regions. It is broken into 3 parts:
1. Querying RIOT for information about summoners, and getting a list of interesting matches.
2. Querying RIOT for the information about those matches.
3. Saving (and potentially loading) those matches.

Load libraries

In [1]:
import requests, json
import numpy as np
from src import API_io
import importlib
import pandas as pd
import time
from src import feature_calc

In [2]:
working_dir = 'C:\\Users\\Me\\Documents\\GitHub\\lolML'
with open(working_dir+ '\\api_key.txt', 'r') as api_file:
    api_key =  api_file.read()

## Get a list of high level matches

Set the region. Then get a JSON of featured games from that region

In [3]:
region_key = 'na'
featured_json = API_io.load_featured_games(api_key, region_key) # load json of featured games
featured_game_ids = [x['gameId'] for x in featured_json ] #  use list comprehension to get featured games; don't use this

Make a list of summoner names and summoner IDs from the featured JSON

In [7]:
summoner_names, summoner_IDs = API_io.get_summoners_IDs_from_featured_games(featured_json, api_key, region_key =region_key)
print(summoner_names[:5],)

['Maki Isuzu Sento', 'Shad0w DTFDragon', 'DZG nothinghere', 'Andybendy', 'OmegaB']


Make a list of match_urls to query, and the request.get params for the season and game type.

In [43]:
match_requests_params = API_io.make_RIOT_request_params(api_key, True, 2015)
summoner_urls = [API_io.make_matchhistory_url(x, region_key =region_key) for x in summoner_IDs]
print(summoner_urls[:2])

['https://na.api.pvp.net/api/lol/na/v2.2/matchlist/by-summoner/23365226', 'https://na.api.pvp.net/api/lol/na/v2.2/matchlist/by-summoner/22352248']


Run the requests

In [41]:
match_histories = [API_io.get_limited_request(x, match_requests_params) for x in summoner_urls ]

Extract the match ID from the match history JSON

In [42]:
match_IDs = API_io.parse_match_json_for_matchIDs(match_histories, region_key)
pd.Series(match_IDs).to_csv(region_key + ' Match IDs.csv')
match_IDs.shape

(31442,)

Convert the match IDs to URLs for querying

In [46]:
match_info_params = API_io.make_RIOT_request_params(api_key, timeline_flag=True)
match_urls = [API_io.make_match_info_url(x, region_key) for x in match_IDs] # True flag means we get the timeline
match_urls[:2]

['https://na.api.pvp.net/api/lol/na/v2.2/match/1704144548',
 'https://na.api.pvp.net/api/lol/na/v2.2/match/1704145839']

## Query RIOT for match information
Determine number of matches to query. Set up the dataframes for querying

In [47]:
match_range = np.arange(0, 10)
col_names = feature_calc.col_names
timeline_end = 55
time_indices = np.arange(5, timeline_end, 5)
games_df = pd.DataFrame(index = match_range, columns= col_names)
timelines_df = [ pd.DataFrame(index = match_range, columns= col_names) for x in time_indices]

Actually run the queries

In [48]:
# this is the worst code EVER
for match_index, cur_match in enumerate(match_range):
    try:
        match_info = requests.get(match_urls[cur_match], match_info_params).json()
        for time_index, last_min in enumerate(time_indices):
            try:
                timelines_df[time_index].loc[match_index] = feature_calc.calc_features_single_match(match_info, last_min)
            except:
                continue
    except requests.exceptions.HTTPError as err:
        print('HHTPError in game ' + str(match_IDs[cur_match]))
    except:
        import sys
        err = sys.exc_info()[0]
        print('Error: ' + str(err) + ' in game ' + str(match_IDs[cur_match]))
    time.sleep(0.3) # RIOT API is throttled to ~0.83 requests / second

In [49]:
timelines_df = [x.dropna() for x in timelines_df]
timelines_df = [feature_calc.retype_columns(x) for x in timelines_df ]
timelines_df = [x.set_index('matchId') for x in timelines_df ]

Check how many matches were downloaded.

In [50]:
[x.shape for x in timelines_df]

[(10, 31),
 (10, 31),
 (10, 31),
 (10, 31),
 (8, 31),
 (7, 31),
 (6, 31),
 (4, 31),
 (1, 31),
 (0, 31)]

If you are scraping a lot of matches in chunks, combine previous matches with new matches.

In [169]:
combined_df = [timelines_df[x].append(combined_df[x] ) for x in np.arange(len(timelines_df)) ]

In [170]:
[x.shape for x in combined_df]

[(9182, 31),
 (9176, 31),
 (9108, 31),
 (8926, 31),
 (7567, 31),
 (5929, 31),
 (3624, 31),
 (1719, 31),
 (571, 31),
 (134, 31)]

### Save the matches to file.

In [174]:
import pickle
with open('combined_df.pickle', 'wb') as pickle_file:
    pickle.dump(combined_df, pickle_file)

In [177]:
with open('combined_df.pickle', 'rb') as pickle_file:
    combined_df = pickle.load(pickle_file)